# Imports

In [ ]:
# pip install autogluon

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from autogluon.tabular import TabularDataset, TabularPredictor

# Load Data

In [2]:
df = pd.read_pickle("../../FantasyData/data-frames/df_basic_ngs_snaps_adv_redzone-tgts_team-snap-data_depth-chart_1999_2023.pkl")

# Filter By Position

In [3]:
df_wr = df.copy().query("position == 'WR'")
print(f"Length of wide receiver data set: {len(df_wr)}")

Length of wide receiver data set: 5039


In [4]:
# remove our na's
df_wr.dropna(subset="Draft Year", inplace=True)
print(f"After removing NA draft rows, data set size: {len(df_wr)}")

After removing NA draft rows, data set size: 4809


In [5]:
df_wr['Draft_Year'] = df_wr['Draft Year'].astype(str).str.replace('s', '')
df_wr["Draft_Year"] = df_wr["Draft_Year"].astype(int)

df_wr["Draft_Round"] = df_wr["Draft Round"].astype(str).str.replace('Undrafted', '8')
df_wr["Draft_Round"] = df_wr["Draft_Round"].astype(int)

df_wr["Draft_Overall"] = df_wr["Draft Overall"].astype(str).str.replace('Undrafted', '400')
df_wr["Draft_Overall"] = df_wr["Draft_Overall"].astype(int)

# WR Relveant Columns

In [6]:
wr_cols = [
    'player_id', 'season',         
    'receptions', 'targets', 'receiving_yards', 'receiving_tds',
    'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
    'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
    'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
    'wopr_x', 'fantasy_points', 'fantasy_points_ppr',
    'games', 'tgt_sh', 'ay_sh', 'yac_sh', 'wopr_y', 'ry_sh', 'rtd_sh',
    'rfd_sh', 'rtdfd_sh', 'dom', 'w8dom', 'yptmpa', 'ppr_sh', 
    'position', 'player_name', 'age', 'team', 'rank', 'tier', 
    'Draft_Year', 'Draft_Round', 'Draft_Overall',
    'Draft Team', 'College', 'avg_cushion', 'avg_separation',
    'avg_intended_air_yards_receiving', 'percent_share_of_intended_air_yards', 
    'catch_percentage', 'yards', 'rec_touchdowns', 'avg_yac', 'avg_expected_yac',
    'avg_yac_above_expectation', 'pfr_player_id', 'offense_snaps', 'offense_pct', 
    'gs_pfr_rec', 'tgt_pfr_rec', 'rec_pfr_rec', 'yds_pfr_rec', 'td_pfr_rec',
    'x1d_pfr_rec', 'ybc_pfr_rec', 'ybc_r_pfr_rec', 'yac_pfr_rec',
    'yac_r_pfr_rec', 'adot_pfr_rec', 'brk_tkl_pfr_rec', 'rec_br_pfr_rec',
    'drop_pfr_rec', 'drop_percent_pfr_rec', 'int_pfr_rec', 'rat_pfr_rec',
    "redzone_targets", "total_snaps", 'Att_passing_totals', 'Att_rushing_totals',
    'depth_team_mean', 'depth_team_min'
]

In [7]:
df_wr = df_wr[wr_cols]

In [8]:
df_wr["yards_per_snap"] = df_wr["receiving_yards"] / df_wr["offense_snaps"]
df_wr["receptions_per_snap"] = df_wr["receiving_yards"] / df_wr["offense_snaps"]

**Note:** Proxy is based on roughly the league median in 2023! This will likely penalize pass happy offenses and improve run happy offenses

Two simplifying assumptions:
 - `freq_pass` is a fair representation of passing across each team in the league
 - `route_proxy` assumes that a WR will be on the field for the same number of rushing and passing plays, proportionally

In [9]:
df_wr["pass_freq"] = df_wr["Att_passing_totals"] / df_wr["total_snaps"]
total_passing_snaps = (df_wr["offense_snaps"] / df_wr["offense_pct"]) * df_wr["pass_freq"]

df_wr["route_proxy"] = total_passing_snaps * df_wr["offense_pct"]
df_wr["yards_per_route_run"] = df_wr["receiving_yards"] / df_wr["route_proxy"]

In [10]:
df_wr["tds_per_redzone_target"] = df_wr["receiving_tds"] / df_wr["redzone_targets"]

**Note:** filtering by snaps automatically removes any data prior to 2012!

In [11]:
df_wr = df_wr.query("games >= 6 and offense_snaps >= 100 and redzone_targets > 0")

In [12]:
len(df_wr)

1520

# Get Lag Version

For having the previous season as features and fantasy points/receiving first downs be the labels.

In [13]:
def create_lag_df(df, cols_to_filter=3, col_to_increment="season", 
                  cols_to_merge=["player_id", "season"]
    ):
    """"""
    df_now = df.copy()
    df_last = df.copy()

    rename_dict = {}
    for col in list(df_last.columns[cols_to_filter:]):
        rename_dict[col] = f"{col}_last"

    df_last.rename(columns=rename_dict, inplace=True)
    df_last[col_to_increment] += 1

    df_lag = df_now.merge(df_last, how='inner', on=cols_to_merge)

    return df_lag

In [14]:
df_lag = create_lag_df(df_wr.copy(), cols_to_filter=2)

In [15]:
df_lag.query("player_name == 'Mike Evans' and season == 2017")[["player_name", "season", 
                                                                "receiving_yards", "receiving_yards_last"]]

,player_name,season,receiving_yards,receiving_yards_last
495,Mike Evans,2017,1001.0,1321.0


In [16]:
import numpy as np

In [17]:
df_lag['improved'] = np.where(df_lag['fantasy_points_ppr'] >= df_lag['fantasy_points_ppr_last'], True, False)

# Select Feature Columns

I.e. the columns with last in them, and of numeric type.

In [18]:
non_numeric_cols = df_lag.select_dtypes(exclude=[np.number]).columns

In [19]:
non_numeric_cols

Index(['player_id', 'position', 'player_name', 'team', 'Draft Team', 'College',
       'pfr_player_id', 'position_last', 'player_name_last', 'team_last',
       'Draft Team_last', 'College_last', 'pfr_player_id_last', 'improved'],
      dtype='object')

In [20]:
assert(df_lag["Draft_Year"].dtype == 'int64')
assert(df_lag["Draft_Round"].dtype == 'int64')
assert(df_lag["Draft_Overall"].dtype == 'int64')

In [21]:
feature_columns = [
    'receptions_last',
    'targets_last',
    'receiving_yards_last',
    'receiving_air_yards_last',
    'receiving_yards_after_catch_last',
    'receiving_first_downs_last',
    'receiving_epa_last',
    'receiving_tds_last',
    # 'receiving_2pt_conversions_last',
    'receiving_fumbles_lost_last',
    'racr_last',
    'target_share_last',
    'air_yards_share_last',
    # 'fantasy_points_last',
    'fantasy_points_ppr_last',
    'games_last',
    'tgt_sh_last',
    'yac_sh_last',
    'w8dom_last',
    'yptmpa_last',
    'ppr_sh_last',
    'age_last',
    'rank_last',
    'tier_last',
    'Draft_Round_last',

    # our metrics
    "yards_per_snap_last",
    "receptions_per_snap_last",
    "route_proxy_last",
    "yards_per_route_run_last",
    "redzone_targets_last",
    "tds_per_redzone_target_last",

    # snap data
    'catch_percentage_last',
    'offense_snaps_last',
    'offense_pct_last',

    'depth_team_mean_last',
    'depth_team_min_last',
    # 'tgt_pfr_rec_last',
    # 'rec_pfr_rec_last',
    # 'td_pfr_rec_last',
    # 'x1d_pfr_rec_last',
    # 'int_pfr_rec_last',
    # 'rat_pfr_rec_last'
]

In [22]:
for col in feature_columns:
    assert(col[-5:] == "_last")

In [23]:
label = "improved"

In [24]:
cols_to_use = feature_columns + [label]
df_ML = df_lag[cols_to_use].copy()

In [25]:
df_ML.head()

,receptions_last,targets_last,receiving_yards_last,receiving_air_yards_last,receiving_yards_after_catch_last,receiving_first_downs_last,receiving_epa_last,receiving_tds_last,receiving_fumbles_lost_last,racr_last,...,route_proxy_last,yards_per_route_run_last,redzone_targets_last,tds_per_redzone_target_last,catch_percentage_last,offense_snaps_last,offense_pct_last,depth_team_mean_last,depth_team_min_last,improved
0,45,59,544.0,589.0,148.0,30.0,27.585154,5,0.0,17.371708,...,328.377923,1.656628,5.0,1.000000,NaN,597.0,0.562667,2.00,2.0,False
1,73,138,1174.0,1832.0,268.0,51.0,25.508378,4,0.0,11.858338,...,468.897059,2.503748,14.0,0.285714,NaN,911.0,0.891250,1.00,1.0,False
2,64,110,745.0,1219.0,178.0,44.0,24.914914,4,0.0,10.475034,...,380.972803,1.955520,16.0,0.250000,NaN,770.0,0.780000,1.00,1.0,True
3,79,134,1065.0,1413.0,363.0,45.0,39.161685,6,1.0,13.027294,...,454.479042,2.343342,20.0,0.300000,NaN,822.0,0.766875,1.00,1.0,False
4,46,73,670.0,719.0,256.0,28.0,22.417599,3,0.0,6.641955,...,221.503305,3.024786,9.0,0.333333,NaN,347.0,0.694286,1.75,1.0,True


In [26]:
for col in df_ML.columns:
    if "fantasy" in col:
        print(col)

fantasy_points_ppr_last


# Create Train, Test, Validation Splits

In [27]:
X = df_ML.iloc[:, :-1]  # features
y = df_ML.iloc[:, -1]   # label

In [28]:
for col in X.columns:
    if "fantasy" in col:
        print(col)

fantasy_points_ppr_last


In [29]:
y[:5]

0    False
1    False
2     True
3    False
4     True
Name: improved, dtype: bool

In [30]:
# First, split into train and temporary sets (train + validation, test)
X_train_temp, X_test, y_train_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Now, split the train_temp into actual train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_temp, y_train_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [31]:
print(f"Length of train set: {len(X_train)}")
print(f"Length of test set: {len(X_test)}")
print(f"Length of val set: {len(X_val)}")

Length of train set: 591
Length of test set: 198
Length of val set: 197


# AutoGluon

In [32]:
train_data = pd.concat([X_train, y_train], axis=1)
val_data = pd.concat([X_val, y_val], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

In [33]:
label_column = 'improved'

## Fit

In [34]:
predictor = TabularPredictor(label=label_column).fit(
    train_data=train_data,
    tuning_data=val_data  # Optional, only if you want to use a separate validation set
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240521_025546"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240521_025546"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #35~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue May  7 09:

## Evaluation

In [35]:
performance = predictor.evaluate(test_data)
print(performance)

{'accuracy': 0.6666666666666666, 'balanced_accuracy': 0.6409161894846434, 'mcc': 0.3050044426225505, 'roc_auc': 0.7122332118688182, 'f1': 0.5416666666666666, 'precision': 0.6610169491525424, 'recall': 0.4588235294117647}


In [36]:
predictions = predictor.predict(test_data.drop(columns=[label_column]))
print(predictions.head())

613    False
451    False
731    False
436    False
275    False
Name: improved, dtype: bool


In [37]:
answers = test_data[label_column]

In [38]:
answers.head()

613    False
451    False
731     True
436     True
275     True
Name: improved, dtype: bool

In [39]:
negative = 0
close = 0
far = 0
for guess, answer in zip(predictions, answers):
    if answer == guess:
        close += 1
    else:
        far += 1

In [40]:
print(f"Correct guesses: {close}")
print(f"Sample space size: {len(answers)}")
print(f"Percent correct: {round(close / len(answers), 3)}")
print(f"Incorrect guesses: {far}")

Correct guesses: 132
Sample space size: 198
Percent correct: 0.667
Incorrect guesses: 66


## Visualization

In [41]:
map_to_player = dict()
j = 0
for i, row in test_data.iterrows():
    player = df_lag.loc[i, 'player_name']
    season = df_lag.loc[i, 'season']
    pair = (player, season)
    map_to_player[j] = pair
    j += 1

In [42]:
correct = list()
miss = list()
for i, (guess, answer) in enumerate(zip(predictions, answers)):
    pair = map_to_player[i]
    pair = (pair[0], pair[1], guess, answer)
    if guess == answer:
        correct.append(pair)
    else:
        miss.append(pair)

In [43]:
miss

[('Cooper Kupp', 2021, False, True),
 ('Odell Beckham', 2015, False, True),
 ('Rueben Randle', 2014, False, True),
 ('Phillip Dorsett', 2019, False, True),
 ('Mike Evans', 2016, False, True),
 ('Adam Thielen', 2018, False, True),
 ('Chester Rogers', 2019, True, False),
 ('Tyreek Hill', 2022, False, True),
 ('DK Metcalf', 2020, False, True),
 ('James Jones', 2014, False, True),
 ('Robert Woods', 2014, False, True),
 ('Marquez Valdes-Scantling', 2020, False, True),
 ('James Jones', 2015, False, True),
 ('Quincy Enunwa', 2016, False, True),
 ('Kenny Stills', 2016, False, True),
 ('Jarius Wright', 2015, True, False),
 ('Cecil Shorts', 2016, True, False),
 ('Davante Adams', 2018, False, True),
 ('Jaelen Strong', 2016, True, False),
 ('Ray-Ray McCloud', 2023, True, False),
 ('Zach Pascal', 2019, False, True),
 ('Dwayne Harris', 2013, True, False),
 ('Danny Amendola', 2015, False, True),
 ('Eric Decker', 2013, False, True),
 ('Jacoby Jones', 2013, False, True),
 ('Kadarius Toney', 2023, True,

## Further Information

In [44]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTreesGini,0.717172,0.639594,accuracy,0.031573,0.026209,0.315299,0.031573,0.026209,0.315299,1,True,8
1,ExtraTreesEntr,0.696970,0.624365,accuracy,0.032459,0.026174,0.286993,0.032459,0.026174,0.286993,1,True,9
2,XGBoost,0.676768,0.624365,accuracy,0.007640,0.002339,0.287602,0.007640,0.002339,0.287602,1,True,10
3,RandomForestEntr,0.671717,0.634518,accuracy,0.030841,0.027585,0.370978,0.030841,0.027585,0.370978,1,True,6
4,CatBoost,0.666667,0.654822,accuracy,0.003146,0.001200,0.349659,0.003146,0.001200,0.349659,1,True,7
5,WeightedEnsemble_L2,0.666667,0.654822,accuracy,0.004607,0.001594,0.405487,0.001461,0.000394,0.055827,2,True,12
6,LightGBMLarge,0.666667,0.624365,accuracy,0.010939,0.002404,0.859821,0.010939,0.002404,0.859821,1,True,11
7,RandomForestGini,0.666667,0.619289,accuracy,0.042084,0.028837,0.401230,0.042084,0.028837,0.401230,1,True,5
8,LightGBM,0.656566,0.649746,accuracy,0.018308,0.001477,0.275770,0.018308,0.001477,0.275770,1,True,4
9,LightGBMXT,0.641414,0.634518,accuracy,0.013301,0.001753,0.319749,0.013301,0.001753,0.319749,1,True,3


For feature clarification:
- **yptmpa:** receiving yards per team pass attempt

In [45]:
predictor.feature_importance(data=test_data)

These features in provided data are not utilized by the predictor and will be ignored: ['receptions_per_snap_last']
Computing feature importance via permutation shuffling for 33 features using 198 rows with 5 shuffle sets...
	1.16s	= Expected runtime (0.23s per shuffle set)
	0.13s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
offense_snaps_last,0.131313,0.019884,0.000061,5,0.172254,0.090372
receptions_last,0.069697,0.012058,0.000103,5,0.094524,0.044870
yac_sh_last,0.029293,0.021840,0.019989,5,0.074262,-0.015676
age_last,0.016162,0.015730,0.041585,5,0.048549,-0.016226
yards_per_route_run_last,0.001010,0.008299,0.399483,5,0.018097,-0.016077
catch_percentage_last,0.000000,0.000000,0.500000,5,0.000000,0.000000
tds_per_redzone_target_last,0.000000,0.000000,0.500000,5,0.000000,0.000000
redzone_targets_last,0.000000,0.000000,0.500000,5,0.000000,0.000000
yptmpa_last,0.000000,0.000000,0.500000,5,0.000000,0.000000
route_proxy_last,0.000000,0.000000,0.500000,5,0.000000,0.000000
